# Model Fine Tuning
----
**Objective**: In this notebook, you will get to fine tune a transformer model on MRPC dataset. You will get to go through the whole process of transfer learning from loading the dataset, loading the model, compiling the model, training the model, and finally evaluating your results.

NOTE: Make sure to change the runtime from CPU to TPU or GPU for faster training

## Install Libraries
Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.5 MB/s eta 0:00:00


## Import Dataset
___
In this section we will use as an example the MRPC (Microsoft Research Paraphrase Corpus) dataset. The dataset consists of 5,801 pairs of sentences, with a label indicating if they are paraphrases or not (i.e., if both sentences mean the same thing).
This is one of the 10 datasets composing the GLUE benchmark, which is an academic benchmark that is used to measure the performance of ML models across 10 different text classification tasks.


**Question 1**: Use the "load_datasets" function to load the "mrpc" dataset from the "glue" benchmark.



In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

## Pre-processing data

First, we need to tokenize our data. We will do so using the BERT tokenizer. We will use the AutoTokenizer with truncation.
This will truncate token by token, removing a token from the longest sequence in the pair if a pair of sequences (or a batch of pairs) is provided.

**Question 2:** Use the .map() function to apply the tokenization functionon the raw dataset. Hint: set the batched flag to True for faster processing.

In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True) # Add your solution here

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Then, we need to pad our data so that all input sequences have the same lengths.

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

Lets take a training example to see what our dataset looks like.

**Question 3:** Display an example from the training section of the dataset. Hint: The tokenized_datasets variable is a nested dictionary where the main keys are "train", "test", and "validate"

In [ ]:
# Add your solution here
train_dataset = tokenized_datasets["train"]
example = train_dataset[0]
print(example)

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0, 'input_ids': [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Finally, we will prepare our training and validation datasets by transforming them into tensorflow datasets.

In [ ]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


## Model Preperation
---
In this section, we will load the model and equip it with a classifier head. This model is already pre-trained and we will fine tune it with the dataset for paraphrasing classification that we prepared.

### Load Classifier Head

In [ ]:
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Compile the model

We will compile the model using SparseCategoricalCrossentropyloss, which is when there are two or more label classes that are not one hot encded.

**Question 4:** Compile the model using the adam optimizer, SparseCategoricalCrossentropy loss (with from_logits flag set to True) and displaying the accuracy metric.

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Add your solution here
model.compile(
    loss = SparseCategoricalCrossentropy(from_logits=True),
    optimizer = 'adam',
    metrics=['accuracy'])

model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Train the model

Now we will train our model using the processing training and validation sets.

**Question 5:** Fit the model on the training and validation sections that we prepared earlier.

In [ ]:
# Add your solution here
model.fit(tf_train_dataset, epochs=5, batch_size=32, validation_data=tf_validation_dataset)

Epoch 1/5
459/459 [==============================] - 151s 203ms/step - loss: 0.6778 - accuracy: 0.6379 - val_loss: 0.6469 - val_accuracy: 0.6838
Epoch 2/5
459/459 [==============================] - 80s 175ms/step - loss: 0.7013 - accuracy: 0.6227 - val_loss: 0.7565 - val_accuracy: 0.3162
Epoch 3/5
459/459 [==============================] - 78s 170ms/step - loss: 0.6931 - accuracy: 0.6183 - val_loss: 0.6240 - val_accuracy: 0.6838
Epoch 4/5
459/459 [==============================] - 78s 170ms/step - loss: 0.6836 - accuracy: 0.6257 - val_loss: 0.6290 - val_accuracy: 0.6838
Epoch 5/5
459/459 [==============================] - 78s 169ms/step - loss: 0.6760 - accuracy: 0.6262 - val_loss: 0.6267 - val_accuracy: 0.6838


## Evaluating the model

Let's test how accurate the models are! In this section, we will predict on the testing dataset to check the model's performance when it comes to classifying paraphrased sentences on unseen data.

In [ ]:
preds = model.predict(tf_test_dataset)["logits"]

216/216 [==============================] - 18s 63ms/step


In [ ]:
import numpy as np
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

(1725, 2) (1725,)


In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=class_preds, references=raw_datasets["test"]["label"])

{'accuracy': 0.664927536231884, 'f1': 0.7987465181058496}